In [151]:
# Import libraries
import pandas as pd
import plotly.express as px
import plotly
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os

# Do not truncate tables
pd.set_option('display.max_columns', None)

In [3]:
# Load the data

def data_path(relative):
    data_source_path = "C:/Users/ljpsm/OneDrive/studie/tue/Visualisation/Assignment/FIFA DataSet/"
    return os.path.join(data_source_path, relative)

# Match data
df_match_data = pd.read_csv(data_path('Data/FIFA World Cup 2022 Match Data/data.csv'), delimiter=',')

# Player data
df_player_defense       = pd.read_csv(data_path('Data/FIFA World Cup 2022 Player Data/player_defense.csv'), delimiter=',')
df_player_gca           = pd.read_csv(data_path('Data/FIFA World Cup 2022 Player Data/player_gca.csv'), delimiter=',')
df_player_keepers       = pd.read_csv(data_path('Data/FIFA World Cup 2022 Player Data/player_keepers.csv'), delimiter=',')
df_player_keepersadv    = pd.read_csv(data_path('Data/FIFA World Cup 2022 Player Data/player_keepersadv.csv'), delimiter=',')
df_player_misc          = pd.read_csv(data_path('Data/FIFA World Cup 2022 Player Data/player_misc.csv'), delimiter=',')
df_player_passing       = pd.read_csv(data_path('Data/FIFA World Cup 2022 Player Data/player_passing.csv'), delimiter=',')
df_player_passing_types = pd.read_csv(data_path('Data/FIFA World Cup 2022 Player Data/player_passing_types.csv'), delimiter=',')
df_player_playingtime   = pd.read_csv(data_path('Data/FIFA World Cup 2022 Player Data/player_playingtime.csv'), delimiter=',')
df_player_possession    = pd.read_csv(data_path('Data/FIFA World Cup 2022 Player Data/player_possession.csv'), delimiter=',')
df_player_shooting      = pd.read_csv(data_path('Data/FIFA World Cup 2022 Player Data/player_shooting.csv'), delimiter=',')
df_player_stats         = pd.read_csv(data_path('Data/FIFA World Cup 2022 Player Data/player_stats.csv'), delimiter=',')

# Team data
df_team_data        = pd.read_csv(data_path('Data/FIFA World Cup 2022 Team Data/team_data.csv'), delimiter=',')
df_team_group_stats = pd.read_csv(data_path('Data/FIFA World Cup 2022 Team Data/group_stats.csv'), delimiter=',')

# Historic data
df_historic_fifa_ranking      = pd.read_csv(data_path('Data/FIFA World Cup Historic/fifa_ranking_2022-10-06.csv'), delimiter=',')
df_historic_matches_1930_2022 = pd.read_csv(data_path('Data/FIFA World Cup Historic/matches_1930_2022.csv'), delimiter=',')
df_historic_world_cup         = pd.read_csv(data_path('Data/FIFA World Cup Historic/world_cup.csv'), delimiter=',')

# Prediction data
df_prediction_groups  = pd.read_csv(data_path('Data/FIFA World Cup 2022 Prediction/2022_world_cup_groups.csv'), delimiter=',')
df_prediction_matches = pd.read_csv(data_path('Data/FIFA World Cup 2022 Prediction/2022_world_cup_matches.csv'), delimiter=',')
df_prediction_international_matches = pd.read_csv(data_path('Data/FIFA World Cup 2022 Prediction/international_matches.csv'), delimiter=',')
df_prediction_world_cup_matches = pd.read_csv(data_path('Data/FIFA World Cup 2022 Prediction/world_cup_matches.csv'), delimiter=',')
df_prediction_world_cups = pd.read_csv(data_path('Data/FIFA World Cup 2022 Prediction/world_cups.csv'), delimiter=',')
    

In [4]:
# Split penalty data from score
df_match_data['score'] = df_match_data['score'].str.replace(r"\(.\)","")

def extract_penalty(score: str, home: bool):
    import re

    if home:
        match = re.match(r'\d+', score)
        if match:
            return match.group()
        else:
            return None
    else:
        match = re.match(r'(\d+)\((\d+)\)', score)
        if match:
            return match.group(2)
        else:
            return None

def extract_score(score: str, home: bool):
    import re

    if home:
        match = re.match(r'\((\d+)\)(\d+)', score)
        if match:
            return match.group(2)
        else:
            return score
    else:
        match = re.match(r'\d+', score)
        if match:
            return match.group()
        else:
            return None
           
# Split match score               
df_match_data[['score_home', 'score_away']] = df_match_data.score.str.split("–", expand=True,)
df_match_data['penalty_home'] = df_match_data['score_home'].str.replace(" ","")
df_match_data['penalty_away'] = df_match_data['score_away'].str.replace(" ","")
df_match_data['penalty_home'] = df_match_data['penalty_home'].apply(lambda x: extract_penalty(x, True))
df_match_data['penalty_away'] = df_match_data['penalty_away'].apply(lambda x: extract_penalty(x, False))
df_match_data['score_home'] = df_match_data['score_home'].str.replace(" ","").apply(lambda x: extract_score(x, True)).astype('int')
df_match_data['score_away'] = df_match_data['score_away'].str.replace(" ","").apply(lambda x: extract_score(x, False)).astype('int')

In [5]:
df_player_gca.columns

Index(['player', 'position', 'team', 'age', 'birth_year', 'minutes_90s', 'sca',
       'sca_per90', 'sca_passes_live', 'sca_passes_dead', 'sca_dribbles',
       'sca_shots', 'sca_fouled', 'sca_defense', 'gca', 'gca_per90',
       'gca_passes_live', 'gca_passes_dead', 'gca_dribbles', 'gca_shots',
       'gca_fouled', 'gca_defense'],
      dtype='object')

### Strong/weak points
compare scoring attempts to actual goal attempts (effectiveness) -> where are they dangerous and where not

df_player_gca:
- gca per 90 / sca          => stats
- gca passes live / sca     => passes
- gca passes dead / sca     => passes
- gca dribbles / sca        => possesion
- gca shots / sca           => shooting
- gca fouled / sca          => misc
- gca defense / sca         => defense

### Shooting

df_player_shooting:
- shots on target per 90
- shots on target pct
- average shooting distance
- goals per shot on target
- shots from free kicks

df_player_stats:
- goals per 90

### Passes

df_player_passing:
- passes completed pct
- assisted shots
- passes into penalty area
- crosses into penalty 

df_player_passing_type:
- crosses
- throw-ins
- corner kicks


### Defensive

df_player_passing_type:
- passes blocked
- clearances
- errros
- % dribblers tackled
- blocks
- interceptions

df_player_misc:
- aerials won / aerials lost


### Miscellaneous

df_player_misc:
- yellow cards
- second yellow cards
- red cards
- fouls comitted / fouls drawn
- offsides
- penalty kicks won / penalty kicks conceded

### Stats

df_player_stats:
- goals per 90
- assists per 90
- pens_made / penalty kicks attempted
- expected goals per 90
- expected assists per 90
- average age of players

### Possesion

df_player_possesion:
- dribbles completed pct
- miscontrols -> make per 90
- dispossessed -> make per 90
- touches in attacking penalty area / touches
- touches in defensive penalty area / touches

# Report

## Domain

User: Coaching staff

Goal: Use visualisation to come up with an effective training strategy/points of improvement for the upcoming math that will take place very soon.

High discriminatory values (literature):
- Possesion
- Total shots
- Shots on goal
- Crosses
- Percentage of succesfull passes


## Data

Tabular data:
- keys: 
    - country name (categorical) -> compare so 2x country names (i.e. 2 items to compare)
    - attribute to compare (categorial) -> 5, 6 or 7
- values (dozens): (quantitative and ordered and normalized) so all continuous values between 0 and 1, normalized on all data

## Tasks:
- Staff should find out where the weak and strong points of both teams differ greatly and hence where opportunities and dangers lie in the upcoming match.
- Staff should be able to compare the weak and strong points of both teams.

## Task abstraction:
- Construct feature (difference weak and strong points between teams) -> identify extremes
- Compare features (weak and strong points of both teams)

## Summary Data & Task

- We have 2 items to compare along dozens of attributes. 
- For which each attribute has a quantitative, ordered and normalized value.
- We want to compare the features, get an overall overview (summarize) and identify extremes.

## Idioms:

Manipulate -> Explore/Filter -> show me something else conditional on where is clicked to limit the number of attributes that are displayed. Can be clustered logically. Otherwise TMI.

Seperate by axis. Align with same scale. Express as a mark on a scale.

Axis orientation Radial / **Parallel**:
- radial because all scales of values are between 0 and 1 and only 5-7 attributes at a time and only 2 teams.
- parallel because radial suggests cyclical nature and angle channel less accurately perceived (tamara boek, p. 166, laatste paragraaf)
- with radial chart it is more difficult to compare two features that are oppositve the circle but might have similar equality/importance/correlation

Do we order the attribute horizontally? Highest difference left, lowest right or something like that?

## Marks & Channels:

Item mark: Line.

Attribute value channel (quantitative): Vertical alligned spatial position.

Key value channel (categorial): spatial region not because spatial already used in attribute encoding, choose second best channel that is color hue.

Set all scales to be between 0 and 1 to compare different feature with each other.

# Code

## Data preparation

In [28]:
# Main chart
strong_weak_points = df_player_gca[['team', 'player']]
strong_weak_points['per90'] = df_player_gca.apply(lambda x: x['gca_per90'] / x['sca_per90'] if x['sca_per90'] > 0 else 0, axis=1)
strong_weak_points['passes_live'] = df_player_gca.apply(lambda x: x['gca_passes_live'] / x['sca_passes_live'] if x['sca_passes_live'] > 0 else 0, axis=1)
strong_weak_points['passes_dead'] = df_player_gca.apply(lambda x: x['gca_passes_dead'] / x['sca_passes_dead'] if x['sca_passes_dead'] > 0 else 0, axis=1)
strong_weak_points['dribbles'] = df_player_gca.apply(lambda x: x['gca_dribbles'] / x['sca_dribbles'] if x['sca_dribbles'] > 0 else 0, axis=1)
strong_weak_points['shots'] = df_player_gca.apply(lambda x: x['gca_shots'] / x['sca_shots'] if x['sca_shots'] > 0 else 0, axis=1)
strong_weak_points['fouled'] = df_player_gca.apply(lambda x: x['gca_fouled'] / x['sca_fouled'] if x['sca_fouled'] > 0 else 0, axis=1)
strong_weak_points['defense'] = df_player_gca.apply(lambda x: x['gca_defense'] / x['sca_defense'] if x['sca_defense'] > 0 else 0, axis=1)
strong_weak_points.drop(['player'], axis=1, inplace=True)
main_chart = strong_weak_points.groupby(by='team').mean()
main_chart = (main_chart - main_chart.min())/(main_chart.max()-main_chart.min()) # normalize
main_chart.reset_index(inplace=True)

C:\Users\ljpsm\AppData\Local\Temp\ipykernel_27884\1495096166.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ljpsm\AppData\Local\Temp\ipykernel_27884\1495096166.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ljpsm\AppData\Local\Temp\ipykernel_27884\1495096166.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [171]:
# stats
stats = df_player_stats[['team', 'goals_per90', 'assists_per90', 'xg_per90', 'xg_assist_per90']]
stats['penality_acc'] = df_player_stats.apply(lambda x: x['pens_made'] / x['pens_att'] if x['pens_att'] > 0 else 0, axis=1)
stats['age'] = df_player_stats.apply(lambda x: int(x['age'].split("-")[0]), axis=1)
stats = stats.groupby(by='team').mean()
stats = (stats - stats.min())/(stats.max()-stats.min()) # normalize -> also normalize age?
stats.reset_index(inplace=True)

C:\Users\ljpsm\AppData\Local\Temp\ipykernel_27884\997916396.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ljpsm\AppData\Local\Temp\ipykernel_27884\997916396.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [178]:
# passes
passes_1 = df_player_passing[['team', 'assisted_shots', 'passes_into_penalty_area', 'crosses_into_penalty_area']]
passes_1['passes_completed_pct'] = df_player_passing.apply(lambda x: x['passes_completed'] / x['passes'] if x['passes'] > 0 else 0, axis=1)
passes_1.set_index('team', inplace=True)
passes_2 = df_player_passing_types[['team', 'crosses', 'throw_ins', 'corner_kicks']].set_index('team')
passes = pd.concat([passes_1, passes_2]).reset_index()
passes = passes.groupby(by='team').mean()
passes = (passes - passes.min())/(passes.max()-passes.min()) # normalize -> also normalize age?
passes.reset_index(inplace=True)

C:\Users\ljpsm\AppData\Local\Temp\ipykernel_27884\3363940414.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [184]:
# possesion
possession = df_player_possession[['team', 'dribbles_completed_pct']]
possession['miscontrols_per90'] = df_player_possession.apply(lambda x: x['miscontrols'] / x['minutes_90s'] if x['minutes_90s'] > 0 else 0, axis=1)
possession['dispossessed_per90'] = df_player_possession.apply(lambda x: x['dispossessed'] / x['minutes_90s'] if x['minutes_90s'] > 0 else 0, axis=1)
possession['pct_touches_att_pen_area'] = df_player_possession.apply(lambda x: x['touches_att_pen_area'] / x['touches'] if x['touches'] > 0 else 0, axis=1)
possession['pct_touches_def_pen_area'] = df_player_possession.apply(lambda x: x['touches_def_pen_area'] / x['touches'] if x['touches'] > 0 else 0, axis=1)
possession = possession.groupby(by='team').mean()
possession = (possession - possession.min())/(possession.max()-possession.min()) # normalize -> also normalize age?
possession.reset_index(inplace=True)

C:\Users\ljpsm\AppData\Local\Temp\ipykernel_27884\3575331387.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ljpsm\AppData\Local\Temp\ipykernel_27884\3575331387.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ljpsm\AppData\Local\Temp\ipykernel_27884\3575331387.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [191]:
# shooting
shooting_1 = df_player_shooting[['team', 'shots_on_target_per90', 'shots_on_target_pct', 'average_shot_distance', 'goals_per_shot_on_target', 'shots_free_kicks']].set_index('team')
shooting_2 = df_player_stats[['team', 'goals_per90']].set_index('team')
shooting = pd.concat([shooting_1, shooting_2]).reset_index()
shooting = shooting.groupby(by='team').mean()
shooting = (shooting - shooting.min())/(shooting.max()-shooting.min()) # normalize -> also normalize age?
shooting.reset_index(inplace=True)

In [194]:
# misc
misc = df_player_misc[['team', 'cards_yellow', 'cards_yellow_red', 'cards_red', 'offsides']]
misc['ratio_fouled_to_fouls'] = df_player_misc.apply(lambda x: x['fouled'] / x['fouls'] if x['fouls'] > 0 else 0, axis=1)
misc['ratio_penalties'] = df_player_misc.apply(lambda x: x['pens_won'] / x['pens_conceded'] if x['pens_conceded'] > 0 else 0, axis=1)
misc = misc.groupby(by='team').mean()
misc = (misc - misc.min())/(misc.max()-misc.min()) # normalize -> also normalize age?
misc.reset_index(inplace=True)

C:\Users\ljpsm\AppData\Local\Temp\ipykernel_27884\895704910.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ljpsm\AppData\Local\Temp\ipykernel_27884\895704910.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [202]:
# defense
defense_1 = df_player_defense[['team', 'blocked_passes', 'clearances', 'errors', 'dribble_tackles_pct', 'blocks', 'interceptions']].set_index('team')
defense_2 = df_player_misc[['team']]
defense_2['ratio_aerials_won_lost'] = df_player_misc.apply(lambda x: x['aerials_won'] / x['aerials_lost'] if x['aerials_lost'] > 0 else 0, axis=1)
defense_2.set_index('team', inplace=True)
defense = pd.concat([defense_1, defense_2]).reset_index()
defense = defense.groupby(by='team').mean()
defense = (defense - defense.min())/(defense.max()-defense.min()) # normalize -> also normalize age?
defense.reset_index(inplace=True)

C:\Users\ljpsm\AppData\Local\Temp\ipykernel_27884\3525316400.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Parallel chart

In [162]:
def parallel_chart(data_frame: pd.DataFrame, *teams_to_compare: list[str]) -> plotly.graph_objs.Figure:
    """Returns a parallel coordinates figure of the dataframe filtered for the teams to compare.
    Automatically scales with numer of teams.
    """
    
    # filter by countries
    df_filtered = data_frame.loc[data_frame['team'].isin(teams_to_compare)]
    teams = df_filtered["team"].astype('category')
    df_filtered['team'] = teams.cat.codes

    # create color scale
    color_scale = []
    for i in range(len(teams_to_compare)):
        color = 255 * i * 1/len(teams_to_compare)
        color_scale.append((i * (1/len(teams_to_compare)), f"rgb({color}, {color}, {color})"))
        color_scale.append(((i+1) * (1/len(teams_to_compare)), f"rgb({color}, {color}, {color})"))

    # create figure with parallel axis (without the categorial team axis)
    columns = [col for col in df_filtered.columns if col != "team"]
    fig = px.parallel_coordinates(df_filtered, 
                              color='team', 
                              color_continuous_scale=color_scale,
                              dimensions=columns
                              )

    # scale dimensions between 0 and 1
    for dimension in fig.data[0]['dimensions']:
        dimension['range'] = [0, 1]

    # include color bar
    fig.update_layout(coloraxis_colorbar=dict(
        title="Teams",
        tickvals=teams.cat.codes,
        ticktext=teams.cat.categories,
        lenmode="pixels", len=100,
        ))

    return fig

In [196]:
parallel_chart(main_chart, 'Argentina', 'France').show()
#parallel_chart(main_chart, 'Argentina', 'France', 'Netherlands').show()

C:\Users\ljpsm\AppData\Local\Temp\ipykernel_27884\131103721.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [190]:
parallel_chart(stats, 'Argentina', 'France').show()

C:\Users\ljpsm\AppData\Local\Temp\ipykernel_27884\131103721.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [180]:
parallel_chart(passes, 'Argentina', 'France').show()

C:\Users\ljpsm\AppData\Local\Temp\ipykernel_27884\131103721.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [185]:
parallel_chart(possession, 'Argentina', 'France').show()

C:\Users\ljpsm\AppData\Local\Temp\ipykernel_27884\131103721.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [189]:
parallel_chart(shooting, 'Argentina', 'France').show()

C:\Users\ljpsm\AppData\Local\Temp\ipykernel_27884\131103721.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [195]:
parallel_chart(misc, 'Argentina', 'France').show()

C:\Users\ljpsm\AppData\Local\Temp\ipykernel_27884\131103721.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [203]:
parallel_chart(defense, 'Argentina', 'France').show()

C:\Users\ljpsm\AppData\Local\Temp\ipykernel_27884\131103721.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

